In [ ]:
import numpy as np
from keras import layers, backend
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras import utils as np_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_uniform 
import scipy.misc
from matplotlib.pyplot import imshow
from keras.initializers import glorot_uniform
from tensorflow import keras
from keras.applications import imagenet_utils
import tensorflow as tf
from __future__ import print_function

import numpy as np
import warnings

In [ ]:
import tensorflow as tf
tf.__version__

!sudo pip3 install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.layers import Layer, InputSpec

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/customxray') #will give the list of folders in the tuber_images

x_ray_types = os.listdir('/content/drive/MyDrive/customxray')
print(x_ray_types)# what types of xray are there in dataset

print("types of xray found:", len(dataset_path))

['Infected', 'Normal']
types of xray found: 2


In [ ]:
rays = []

for item in x_ray_types:
    all_rays = os.listdir('/content/drive/MyDrive/customxray' + '/' + item)
    
    
    for ray in all_rays:
        rays.append((item, str('/content/drive/MyDrive/customxray' + '/' + item) + '/' + ray))
        #print(rays)
rays

[('Infected',
  '/content/drive/MyDrive/customxray/Infected/person1664_virus_2877.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person20_virus_51.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person150_bacteria_717.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person1662_virus_2875.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person35_virus_80.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person157_bacteria_740.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person136_bacteria_648.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person3_virus_15.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person111_bacteria_536.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person152_bacteria_723.jpeg'),
 ('Infected',
  '/content/drive/MyDrive/customxray/Infected/person1635_virus_2831.jpeg'),
 ('Infected',
  '/

In [ ]:
rooms_df = pd.DataFrame(data=rays, columns=['x_ray types', 'image'])
print(rooms_df.head())

  x_ray types                                              image
0    Infected  /content/drive/MyDrive/customxray/Infected/per...
1    Infected  /content/drive/MyDrive/customxray/Infected/per...
2    Infected  /content/drive/MyDrive/customxray/Infected/per...
3    Infected  /content/drive/MyDrive/customxray/Infected/per...
4    Infected  /content/drive/MyDrive/customxray/Infected/per...


In [ ]:
print("Total number of images in dataset: ", len(rooms_df))

room_count = rooms_df['x_ray types'].value_counts()

print("xrays in each category: ")
print(room_count)

Total number of images in dataset:  4846
xrays in each category: 
Normal      3734
Infected    1112
Name: x_ray types, dtype: int64


In [ ]:
import cv2
path = '/content/drive/MyDrive/customxray/'

im_size=224

images = []
labels=[]

for i in x_ray_types:
    data_path = path +str(i)
    filenames = [i for i in os.listdir(data_path)]
    
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)
labels

['Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',
 'Infected',

In [ ]:
images = np.array(images)


images.shape

(4846, 224, 224, 3)

In [ ]:
images = images.astype('float32')/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
y=rooms_df['x_ray types'].values

y_labelencoder = LabelEncoder()
y= y_labelencoder.fit_transform (y)
print(y)

[0 0 0 ... 1 1 1]


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images,y = shuffle(images,y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split( images, y, test_size=0.30, random_state=415)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(3392, 224, 224, 3)
(3392,)
(1454, 224, 224, 3)
(1454,)


In [ ]:
def identity_block(X, f, filters):
  #retrive filters
  F1, F2, F3 = filters

  X_shortcut = X


#first layer
  X= Conv2D(filters = F1, kernel_size=(1,1), strides = (1,1), padding = 'same')(X)
  X= BatchNormalization(axis=3)(X)
  X= Activation('relu')(X)

  #second layer
  X= Conv2D(filters = F2, kernel_size=(f,f), strides = (1,1), padding = 'same')(X)
  X= BatchNormalization(axis=3)(X)
  X= Activation('relu')(X)

  #third layer
  X= Conv2D(filters = F3, kernel_size=(1,1),strides = (1,1), padding = 'same')(X)
  X= BatchNormalization(axis=3)(X)


  #final step: adding shoetcut value to F(x), and pass it through Relu activation
  X= Add()([X, X_shortcut])
  X= Activation('relu')(X)

  return X

In [ ]:
def convolution_block(X, f, filters, s=2):
  #retrive filters
  F1, F2, F3 = filters
#saving the input value
  X_shortcut = X

#first layer
  X= Conv2D(F1, (1,1), strides = (s,s))(X) #1,1 is a filter size
  X= BatchNormalization(axis=3)(X)
  X= Activation('relu')(X)

#second layer (f,f)=3*3 filter by default
  X= Conv2D(filters = F2, kernel_size=(f,f), strides = (1,1), padding = 'same')(X)
  X= BatchNormalization(axis=3)(X)
  X= Activation('relu')(X)

#third layer
  X= Conv2D(filters = F3, kernel_size=(1,1),strides = (1,1), padding = 'same')(X)
  X= BatchNormalization(axis=3)(X)

#shortcutpath
  X_shortcut= Conv2D(filters = F3, kernel_size=(1,1),strides = (s,s), padding = 'same')(X_shortcut)
  X_shortcut= BatchNormalization(axis=3)(X_shortcut)


  #final step: adding shoetcut value to F(x), and pass it through Relu activation
  X= Add()([X, X_shortcut])
  X= Activation('relu')(X)


  return X

In [ ]:
#from keras.engine.input_layer import InputLayer
def ResNet50(input_shape=(224,224,3), classes=2):

  
  #defining the input with shape input_shape
  X_input = Input(input_shape)
  #keras.layers.Input(input_shape=(224, 224, 3))
  #zero padding
  x = ZeroPadding2D((3,3))( X_input) #3,3 padding

  #stage1
  x= Conv2D(64,(7,7), strides = (2,2))(x) #1,1 is a filter size
  x= BatchNormalization(axis=3)(x)
  x= Activation('relu')(x)
  x= MaxPooling2D((3,3), strides=(2,2))(x)


#second layer
  x = convolution_block(x, f=3, filters=[64,64,256], s=1)
  x = identity_block(x, 3, [64,64,256])
  x = identity_block(x, 3, [64,64,256])

#third stage
  x = convolution_block(x, f=3, filters=[128,128,512], s=2)
  x = identity_block(x, 3, [128,128,512])
  x = identity_block(x, 3, [128,128,512])  
  x = identity_block(x, 3, [128,128,512])

#fourth stage
  x = convolution_block(x, f=3, filters=[256,256,1024], s=2)
  x = identity_block(x, 3, [256,256,1024])
  x = identity_block(x, 3, [256,256,1024])  
  x = identity_block(x, 3, [256,256,1024])
  x = identity_block(x, 3, [256,256,1024])
  x = identity_block(x, 3, [256,256,1024])

#fifth stage
  x = convolution_block(x, f=3, filters=[512,512,2048], s=2)
  x = identity_block(x, 3, [512,512,2048])
  x = identity_block(x, 3, [512,512,2048])

#average pool
  x = AveragePooling2D((2,2),name="avg_pool")(x)

#output layer
  x = Flatten()(x)
  x = Dense(classes, activation='softmax',name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=42))(x)

#creating a model
  model = Model(inputs = X_input, outputs=x, name='ResNet50')

  return model

In [ ]:
model = ResNet50(classes=2)

In [ ]:
model.compile(optimizer='adam', loss='categorial_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_53 (Conv2D)             (None, 112, 112, 64  9472        ['zero_padding2d_1[0][0]']       
                                )                                                          

In [ ]:
#model.compile(optimizer = 'adam', loss='categorial_crossentropy', metrics=['accuracy'])
model.compile(
  loss='SparseCategoricalCrossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
model.fit(train_x, train_y, epochs = 5, batch_size=32)

Epoch 1/5
106/106 [==============================] - 272s 3s/step - loss: 1.1471 - accuracy: 0.8552
Epoch 2/5
106/106 [==============================] - 266s 3s/step - loss: 0.3200 - accuracy: 0.9145
Epoch 3/5
106/106 [==============================] - 267s 3s/step - loss: 0.2378 - accuracy: 0.9281
Epoch 4/5
106/106 [==============================] - 266s 3s/step - loss: 0.1478 - accuracy: 0.9472
Epoch 5/5
106/106 [==============================] - 269s 3s/step - loss: 0.1296 - accuracy: 0.9558


In [ ]:
preds = model.evaluate(test_x, test_y)
print("loss = " + str(preds[0]))
print("Test accuracy= " + str(preds[1]))

46/46 [==============================] - 24s 494ms/step - loss: 0.0945 - accuracy: 0.9684
loss = 0.09449135512113571
Test accuracy= 0.9683631658554077
